# Some Puzzle Archeology: Catching the Mice

  * John Mount
  * Nina Zumel
  * https://www.win-vector.com
  * October 11, 2024

## Introduction

Nina Zumel shared another puzzle from Henry Dudeney’s article “The World’s Best Puzzles”, *The Strand Magazine*, December 1908 with me (John). The puzzle is "Catching the Mice", and has a very mathematical flavor. We both became a bit obsessed with it. In particular we became obsessed with working out how to solve this using pencil and paper, as one would have in 1908. 

We thought we had a pretty slick approach, and we got it to work at a comfortable pencil and paper level. Then the published solution came off as a bit of a shot across our bow.

We are experimenting in how to explain 1908 technical methods to a 2024 technical audience. Our current didactic idea is to translate into Python code.

The puzzle is as follows.

<center>
<img src="mouse_puzzle.png">
</center>

This is a type of puzzle that is actually quite fun to code up and brute force enumerate all the solutions of. It shows what a miracle having access to programmable calculating machines is. We can also try to use math and pencil and paper style work to solve it.

Puzzle expert Peter Winkler knows this as a variation of the [Josephus puzzle](https://en.wikipedia.org/wiki/Josephus_problem) (by the way, [Peter Winkler's *Mathematical Puzzles: Revised Edition*](https://www.amazon.com/Mathematical-Puzzles-Peters-Recreational-Mathematics/dp/1032708484/) is a **must** own; the puzzles are *much* better than most collections and the teaching in the solutions and hints sections is amazing).

## Capturing the question

Let's translate the problem into one of items in a list.

We find it a bit easier to work in terms of "how many mice we skip" instead "every kth mouse." For example: eating every other mouse corresponds to skipping one mouse after each consumption. So all of our displacement numbers are one less than claimed in the article. We will also put the white mouse in position 0 instead of 1 (again a notational shift of 1), as that is more compatible with Python's data conventions.

The function below instantiates the following procedure.

  * 13 mice are labeled and put in a list numbered `0` through `12`.
  * The white mouse is placed in position `0`.
  * The cat pointer starts pointing at position `p = start`.
  * The cat pointer repeats `k` times:
    * Move from position `p` to position `(p + advance) % len(list)`.
    * Delete the mouse at position `p` and shift all the mice in positions higher than `p` one down to get a new shorter contiguous list of mice.

In the original system a forward motion or `advance` of `995` would look like the following.

<center>
<img src="one_step_orig.png" height=500>
</center>

For our list version the same `advance = 995` move is represented by moving to the 7th item in the list, deleting that mouse, and shifting the higher numbered mice down.

<center>
<img src="one_step_list.png" height=500>
</center>

In Python the list procedure is captured as the function `run_cat_process()`.

In [1]:
# imports
import inspect
import numpy as np
import pandas as pd
from IPython.display import Code, display
from catching_the_mice_fns import (
    check_soln,
    create_mod_k_column,
    push_column_back,
    run_cat_process,
    sieve_solutions_11_12_13,
    WHITE_MOUSE,
)

In [2]:
# show the source code for the run_cat_process()
display(Code(inspect.getsource(run_cat_process), language="python"))

def run_cat_process(*, start: int, advance: int, k: int) -> Tuple[int, ...]:
    """
    Work out what mice are eaten in k repetitions of advancing advance mice forward
    in  list (wrapping around) and then removing the mouse pointed to. Initial start
    is at start index.
    """
    eaten = []
    mice = [WHITE_MOUSE] + [f"black mouse {i+1}" for i in range(12)]
    p = start
    for _ in range(k):
        p = (p + advance) % len(mice)  # perform the move
        v = mice[p]  # get the label of the mouse now pointing at
        mice.remove(v)  # remove from the list (and shift remaining mice)
        eaten.append(v)  # add removed mouse to ordered removal list
    return tuple(eaten)

A puzzle solver would not implement this code. They would simulate the procedure in a diagram or with markers. The code is trying to capture and (over) describe what such a solver may be doing.

We confirm that, as the puzzle introduction states, starting at the seventh mouse and consuming every 13th mouse will consume the white mouse last.  Remember that our notation is one off from Dudeney's notation (so we start at the mouse numbered 6, and advance 12 mice).

In [3]:
# confirm white mouse eaten last claim
traj_12 = run_cat_process(start=7 - 1, advance=13 - 1, k=13)
assert len(traj_12) == 13
assert len(set(traj_12)) == len(traj_12)
assert traj_12[12] == WHITE_MOUSE

traj_12

('black mouse 5',
 'black mouse 6',
 'black mouse 8',
 'black mouse 11',
 'black mouse 2',
 'black mouse 10',
 'black mouse 7',
 'black mouse 9',
 'black mouse 1',
 'black mouse 3',
 'black mouse 4',
 'black mouse 12',
 'white mouse')

## Solving the problem 1908 style

Let's translate the "catching the mice" procedure into a set of equations that are valid if and only if the mouse is eaten on the 3rd step. `advance` will be the number of steps forward the cat takes at each step, and we start at mouse `0`.

It is always a good idea to try simpler problem variations. Let's try simpler problems and look for invariants (another good puzzle strategy).

### Eating the white mouse on the 1st move

To eat the mouse in the first move: the cat must have an `advance` that net runs around the circle `+13` steps. This is exactly:

  * `(advance % 13) = 0`.

In [4]:
# make a table of check expression values and whether the start is a solution to the "catch the mouse on 1st step problem"
table_1 = pd.DataFrame({
    'advance': [6, 13, 25, 26, 31]
})
table_1['(advance % 13)'] = table_1['advance'] % 13
table_1['obeys check equations'] = table_1['(advance % 13)'] == 0
table_1['run results'] = [run_cat_process(start=0, advance=advance, k=1) for advance in table_1['advance']]
table_1['mouse eaten on 1st step'] = [v[0] == WHITE_MOUSE for v in table_1['run results']]

table_1

,advance,(advance % 13),obeys check equations,run results,mouse eaten on 1st step
0,6,6,False,"(black mouse 6,)",False
1,13,0,True,"(white mouse,)",True
2,25,12,False,"(black mouse 12,)",False
3,26,0,True,"(white mouse,)",True
4,31,5,False,"(black mouse 5,)",False


### Eating the white mouse on the 2nd move

To eat the mouse in the second move, the cat must not eat the white mouse in the first move, and then must eat the mouse in the zeroth position at the end of the second move. This means the cat's net motion around the original 13 mice plus the remaining 12 mice looks like a net-0 motion. In equations this is as follows.

  * `(advance % 13) != 0`
  * `((advance % 13) + advance) % 12 = 0`.

It is worth working an example to get comfortable with this.

In [5]:
# make a table of check expression values and if the start is solution to the "catch the mouse on 2nd step problem"
table_2 = pd.DataFrame({
    'advance': [6, 13, 25, 26, 31]
})
table_2['(advance % 13)'] = table_2['advance'] % 13
table_2['((advance % 13) + advance) % 12'] = ((table_2['advance'] % 13) + table_2['advance']) % 12
table_2['obeys check equations'] = (
    (table_2['(advance % 13)'] != 0) 
    & (table_2['((advance % 13) + advance) % 12'] == 0))
table_2['run results'] = [run_cat_process(start=0, advance=advance, k=2) for advance in table_2['advance']]
table_2['mouse eaten on 2nd step'] = [v[1] == WHITE_MOUSE for v in table_2['run results']]

table_2

,advance,(advance % 13),((advance % 13) + advance) % 12,obeys check equations,run results,mouse eaten on 2nd step
0,6,6,0,True,"(black mouse 6, white mouse)",True
1,13,0,1,False,"(white mouse, black mouse 2)",False
2,25,12,1,False,"(black mouse 12, black mouse 1)",False
3,26,0,2,False,"(white mouse, black mouse 3)",False
4,31,5,0,True,"(black mouse 5, white mouse)",True


Here is where we see some of the cleverness of the puzzle. Solutions to eating the mouse can be built up by partitioning the number of positions to advance into smaller numbers. For example our first solution is partitioning `12` into `6 + 6`. The possible surprise is: many solutions are in fact much larger than the number of items in the diagram.

### Eating the white mouse on the 3rd move

To check if the mouse is eaten on the 3rd move, we again must not obey the earlier check equation and we must obey a new check saying 3 moves end in position zero. These check equations turn out to be the following.

  * `(advance % 13) != 0`
  * `((advance % 13) + advance) % 12 != 0`
  * `(((advance % 13) + advance) % 12 + advance) % 11 = 0`.

In [6]:
# make a table of check expression values and whether the start is a solution to the "catch the mouse on 3rd step problem"
table_3 = pd.DataFrame({
    'advance': [6, 13, 25, 26, 31]
})
table_3['(advance % 13)'] = table_3['advance'] % 13
table_3['((advance % 13) + advance) % 12'] = ((table_3['advance'] % 13) + table_3['advance']) % 12
table_3['(((advance % 13) + advance) % 12 + advance) % 11'] = (((table_3['advance'] % 13) + table_3['advance']) % 12 + table_3['advance']) % 11
table_3['obeys check equations'] = (
    (table_3['(advance % 13)'] != 0) 
    & (table_3['((advance % 13) + advance) % 12'] != 0) 
    & (table_3['(((advance % 13) + advance) % 12 + advance) % 11'] == 0))
table_3['run results'] = [run_cat_process(start=0, advance=advance, k=3) for advance in table_3['advance']]
table_3['mouse eaten on 3rd step'] = [v[2] == WHITE_MOUSE for v in table_3['run results']]

table_3

,advance,(advance % 13),((advance % 13) + advance) % 12,(((advance % 13) + advance) % 12 + advance) % 11,obeys check equations,run results,mouse eaten on 3rd step
0,6,6,0,6,False,"(black mouse 6, white mouse, black mouse 8)",False
1,13,0,1,3,False,"(white mouse, black mouse 2, black mouse 5)",False
2,25,12,1,4,False,"(black mouse 12, black mouse 1, black mouse 5)",False
3,26,0,2,6,False,"(white mouse, black mouse 3, black mouse 8)",False
4,31,5,0,9,False,"(black mouse 5, white mouse, black mouse 11)",False


## Sieve Search

Now that we have reduced the problem to satisfying a system of equations, a standard technique of the time would be a [sieve method](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes). The idea of a sieve is to build a table a lot like the ones we have just seen, but incrementally using only simple addition and subtraction. The source code for our sieve is in the appendix. Translating the check equations into a sieve process should have been both familiar and easy for Dudeney and his more mathematical readers (obviously they could not translate into Python). The sieve method is, by design, a very easy task on pencil and paper. Below is the sieve table extended out to 200 different integers. Dudeney could have quickly calculated this by hand.

In [7]:
# build up the annotations that tells us what is a solution and what is not
sieve = sieve_solutions_11_12_13(200)

sieve.head()

,advance,advance%11,advance%12,advance%13,((advance % 13) + advance) % 12,(((advance % 13) + advance) % 12) + advance) % 11,is_sieve_solution
0,0,0,0,0,0,0,False
1,1,1,1,1,2,3,False
2,2,2,2,2,4,6,False
3,3,3,3,3,6,9,False
4,4,4,4,4,8,1,False


In [8]:
# show only the solution rows
sieve.loc[sieve["is_sieve_solution"], :]

,advance,advance%11,advance%12,advance%13,((advance % 13) + advance) % 12,(((advance % 13) + advance) % 12) + advance) % 11,is_sieve_solution
99,99,0,3,8,11,0,True
103,103,4,7,12,7,0,True
111,111,1,3,7,10,0,True
115,115,5,7,11,6,0,True
123,123,2,3,6,9,0,True
127,127,6,7,10,5,0,True
135,135,3,3,5,8,0,True
139,139,7,7,9,4,0,True
147,147,4,3,4,7,0,True
151,151,8,7,8,3,0,True


The smallest solution we found was `advance = 99`. This solution is surprisingly large, and probably part of the cleverness of the puzzle.

As a side note. A good math puzzle discipline is: "look" followed "conjecture then prove or disprove." We notice `advance % 12` is only `3` or `7` for the found examples. We will look at this in the appendix, but it turns out to not (on its own) to be a puzzle invariant.

## Confirming our solution

Since we do have a computer, we can confirm the sieve method found exactly the solutions in the range of interest. Dudeney likely would not perform this check at this scale. Likely he would perform smaller checks and be careful with his figuring.

In [9]:
sieve["is_brute_force_solution"] = [
    check_soln(run_cat_process(start=0, advance=advance, k=3)) for advance in sieve["advance"]
]
assert np.all(sieve["is_sieve_solution"] == sieve["is_brute_force_solution"])

Yey! we got everything right. Let's look at the published solution.

## The official solution

The official solution (from a later edition of the Strand) is just the following snippet.

<img src="mouse_answer.png">



What the heck! Did Dudeney really sieve up to `1000`?

Sieving up to `100` would have been easy for Dudeney and presumably for many of his math aware readers. Sieving up through `1000` to find a `99` seems unlikely. Is Dudeney signalling to his readers that he has a much flashier method than pedestrian sieving? Did we all just get clowned on?

And the solution detail: "there are just seventy-two other numbers between these that the cat might employ with equal success; but she would select the smallest" could be a feint giving the appearance that Dudeney is using a method that doesn't present solution candidates in order. Sieve methods are ordered, so this would appear to be a further denial of sieving for the solution.

Our sieve table can immediately confirm *some* of the claims immediately. `100 - 1` is indeed a solution and the smallest solution (which was what was asked for!). 


In [10]:
assert sieve.loc[sieve["advance"] == 100 - 1, "is_sieve_solution"].values[0] == True
assert np.min(sieve.loc[sieve["is_sieve_solution"], "advance"]) == 100 - 1


It is also **trivial** for *us* to sieve up through `1000` on our computer, to confirm the rest of the published claims. However, we are now motivated to find a solution method that would let Dudeney eliminate on the order of `1000` candidates by hand without excessive effort. We have in fact found one, and the work to develop it is the topic of [our follow up note](catching_the_mice_modular.ipynb).



## Conclusion

We solved the problem using a sieve technique that would be easy for to implement using pencil and paper. The method involves only addition and subtraction of small numbers.

In our [next note](catching_the_mice_modular.ipynb), we do the brutal work of developing a modular arithmetic solution that moves over groups of related solutions (instead of needing to sieve candidates or even individual solutions). This again will be something that could be done using pencil and paper- but something that requires a good number of preparatory lemmas.

## More in this mathematical puzzles series

This puzzle turned out to be a bit harder than we anticipated. More friendly puzzles in this series include:

  * [100 Bushels of Corn](https://ninazumel.com/blog/2024-09-26-100bushels/)
  * [Solving 100 Bushels Using Matrix Factorization](https://win-vector.com/2024/09/29/solving-100-bushels-using-matrix-factorization/)
  * [Bachet’s Four Weights Problem](https://ninazumel.com/blog/2024-09-29-four-weights/)
  * [The Perplexed Banker](https://ninazumel.com/blog/2024-10-03-perplexed-banker/)
  * [Dudeney’s Remainder Problem](https://win-vector.com/2024/10/06/dudeneys-remainder-problem/)
  * [Coin Puzzles](https://ninazumel.com/blog/2024-10-08-coin-puzzles/)
  * [The Wine Thief Problem](https://ninazumel.com/blog/2024-10-10-wine-thief/)

## Appendix

### The sieve code

A Python realization of the sieve procedure is given below. By design, the sieve involves only steps such as adding non-negative integers no larger than `26` and subtracting `11` or `12` a small number of times.

In [11]:
# show the source code for the sieve process
for function in [create_mod_k_column, push_column_back, sieve_solutions_11_12_13]:
    display(Code(inspect.getsource(function), language="python")) 

def create_mod_k_column(m_range: int, *, mod: int):
    """Write (i % mod) for i in range(m_range) without division or explicit remainders"""
    modulo_count = np.zeros(m_range, dtype=int)
    row_i = 0
    remainder = 0
    while row_i < m_range:
        modulo_count[row_i] = remainder
        # prepare for next pass
        row_i += 1
        remainder += 1
        if remainder >= mod:
            remainder = 0
    return modulo_count

def push_column_back(c, *, ub: int):
    """round a vector of non-negative integers into range [0, ub-1] with only subtractions."""
    c = c.copy()  # don't alter original
    while True:  # this loop will only execute one or twice for the range of numbers we are giving it
        overs = c >= ub
        if not np.any(overs):
            return c
        c[overs] -= ub

def sieve_solutions_11_12_13(m_range: int) -> pd.DataFrame:
    """
    Build up a sieving table for "catching the mice".
    The goal is to describe a method that would be effective with pencil and paper.
    The emphasis is on avoiding many steps and avoiding expensive steps.
    """
    sieve = {"advance": range(m_range)}
    for mod in (11, 12, 13):
        sieve[f"advance%{mod}"] = create_mod_k_column(m_range, mod=mod)
    # calculate derived check equation columns
    sieve["((advance % 13) + advance) % 12"] = push_column_back(
        sieve["advance%13"] + sieve["advance%12"], ub=12)
    sieve["(((advance % 13) + advance) % 12) + advance) % 11"] = push_column_back(
        sieve["((advance % 13) + advance) % 12"] + sieve["advance%11"], ub=11)
    sieve["is_sieve_solution"] = (
        (sieve["advance%13"] != 0)
        & (sieve["((advance % 13) + advance) % 12"] != 0)
        & (sieve["(((advance % 13) + advance) % 12) + advance) % 11"] == 0))
    return pd.DataFrame(sieve)

### The `advance%12 = 3 or 7` conjecture

When examining the problem we noticed the following possible regularity. Looking, conjecturing, and then proving a critical puzzle methods. Let's look at the "3-7 conjecture. 

In [12]:
sieve.loc[sieve["is_sieve_solution"], ['advance', 'advance%12', 'is_brute_force_solution']]

,advance,advance%12,is_brute_force_solution
99,99,3,True
103,103,7,True
111,111,3,True
115,115,7,True
123,123,3,True
127,127,7,True
135,135,3,True
139,139,7,True
147,147,3,True
151,151,7,True


Wow, the 3-7 conjecture stands out and even appears to have a regular alternating structure.

Let's re-check this for `advance` in the range `[0, 11 * 12 * 13 = 1716)` (`1716` being the least common multiplier of our moduli, and where puzzle solutions start systematically repeating).

In [13]:
ub = 11 * 12 * 13
twelve_conjecture_frame = pd.DataFrame({
    'advance%12': [advance % 12 for advance in range(ub)],
    'is puzzle solution': [run_cat_process(start=0, advance=advance, k=3)[2] == WHITE_MOUSE for advance in range(ub)],
    'advance%12 = 3 or 7': [(advance % 12) in [3, 7] for advance in range(ub)]
})

The `advance%12 = 3 or 7` holds for quite a range of solutions.

In [14]:
first_violation = np.where(twelve_conjecture_frame['is puzzle solution'] & (twelve_conjecture_frame['advance%12 = 3 or 7'] == False))[0][0]

first_violation

298

In [15]:
pd.crosstab(
    twelve_conjecture_frame.loc[range(first_violation), 'is puzzle solution'],
    twelve_conjecture_frame.loc[range(first_violation), 'advance%12 = 3 or 7']
)

advance%12 = 3 or 7,False,True
is puzzle solution,,
False,248,35
True,0,15


The lower left cell of the cross tabulation table is exactly the claim `"is puzzle solution" -> "advance%12 = 3 or 7"`.

But this implication is not in fact true for all solutions.

In [16]:
pd.crosstab(
    twelve_conjecture_frame['is puzzle solution'],
    twelve_conjecture_frame['advance%12 = 3 or 7']
)

advance%12 = 3 or 7,False,True
is puzzle solution,,
False,1320,264
True,110,22


The conjecture holding for such a long and regular interval (all integers from `0` through `297`, which includes the first `15` puzzle solutions) really tricks one into believing it. For simple puzzles, it is in fact a bit unusual for something to hold for so long and then fail. However, "catching the mice" is carrying state (position and how many mice remain) and carrying state can cause very subtle long term interactions. This is why we have to prove things instead of merely claiming them.